In [8]:
using CSV, DataFrames

In [9]:
primAuthor = "Marin"
fileOwnPapers = "MCN_JL_savedrecs_mine.txt"
usingPART = false
if usingPART
    fileCitations = "LS_JL_savedrecs_PART.txt"
else
    fileCitations = "MCN_JL_savedrecs.txt"
end

"MCN_JL_savedrecs.txt"

In [10]:
fileOwnCSV  = "MCN_JL_MINE_test.csv"
fileOwnTXT  = "MCN_JL_total.txt"
fileOwnUAIC = "MCN_JL_UAIC1.txt"

"MCN_JL_UAIC1.txt"

In [11]:
#root = dirname(@__FILE__)
root = dirname("/mnt/e/Stoleriu/Doc/CV/auto/")
inputAIS = joinpath(root, "AI_factor_full_1997-2022.txt")
inputISI = joinpath(root, "IF_factor_full_1997-2022.txt")
AIFirstYear = 1997
AILastYear = 2022

2022

Pentru calculele pe 5 ani (sau orice alta durata):

In [12]:
ANini = 1999
ANfin = 2022

2022

In [13]:
AIS = CSV.read(inputAIS, DataFrame; header=true, delim='\t')
AIS[8157, [1, 2, 3, 14, 25, 26, 27]]

Row,Journal,1997,1998,2009,2020,2021,2022
,String31,Float64,Float64,Float64,Float64,Float64,Float64
8157,IEEE INTEL TRANSP SY,0.356,0.356,0.356,0.989,1.031,1.031


In [14]:
#VSCodeServer.vscodedisplay(AIS)

In [15]:
ISI = CSV.read(inputISI, DataFrame; header=true, delim='\t')
ISI[8157, [1, 2, 3, 14, 25, 26, 27]]

Row,Journal,1997,1998,2009,2020,2021,2022
,String31,Float64,Float64,Float64,Float64,Float64,Float64
8157,IEEE INTEL TRANSP SY,0.821,0.821,0.821,3.419,5.293,5.293


In [16]:
#root = dirname(@__FILE__)
#root = dirname("/mnt/e/Stoleriu/Doc/CV/auto/")
inputMINE = joinpath(root, fileOwnPapers)
inputCIT = joinpath(root, fileCitations)

"/mnt/e/Stoleriu/Doc/CV/auto/MCN_JL_savedrecs.txt"

In [17]:
MINEraw = CSV.read(inputMINE, DataFrame; header=true, delim='\t')
sort!(MINEraw, [:PY], rev = true)
eltype.(eachcol(MINEraw));

In [18]:
#VSCodeServer.vscodedisplay(MINEraw)

In [19]:
#MINE = MINEraw[:, [2, 9, 24, 30, 32, 42, 45, 46, 47, 52, 53, 54, 55]];
#MINE = MINEraw[:, [2, 9, 24, 32, 42, 45, 46, 47, 52, 53, 54, 55]];
#MINE = MINEraw[:, [2, 9, 25, 32, 43, 46, 47, 53, 54, 55, 56]];
#                  AU TI  RP  NR  J9  PY  VL  BP  EP  AR  DI
MINE = MINEraw[:, [2, 9, 25, 33, 44, 47, 48, 54, 55, 56, 57]];  # fara citari (col 30)
eltype.(eachcol(MINE));

In [20]:
#MINE[2, :PY] = 2021   # ADV MAT no year!!!!
eltype.(eachcol(MINE));

In [21]:
#VSCodeServer.vscodedisplay(MINE)

Pregatim MINE cu noi coloane

In [22]:
rename!(MINE,:"J9" => :"Journal")
insertcols!(MINE, :NA => 0);
insertcols!(MINE, :NEFF => 0.0);
insertcols!(MINE, :PRIM => 0);
insertcols!(MINE, :AI => 0.0);
insertcols!(MINE, :AINEFF => 0.0);
insertcols!(MINE, :IF => 0.0);
insertcols!(MINE, :IFNA => 0.0);
insertcols!(MINE, :TIMESC => 0);
insertcols!(MINE, :CT => 0.0);
### 5 ani
insertcols!(MINE, :PRIM5 => 0);
insertcols!(MINE, :AINEFF5 => 0.0);
insertcols!(MINE, :IFNA5 => 0.0);
insertcols!(MINE, :TIMESC5 => 0);
insertcols!(MINE, :CT5 => 0.0);
### UAIC
insertcols!(MINE, :UAIC1 => 0.0);
insertcols!(MINE, :UAIC9 => 0.0);

In [23]:
#VSCodeServer.vscodedisplay(MINE)

Gasim numarul de autori si prim autor

In [24]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    MINE[row_index, :NA] = (1+count(";", lucrare[:AU]))

    if MINE[row_index, :NA] <= 5
        MINE[row_index, :NEFF] = MINE[row_index, :NA]
    elseif MINE[row_index, :NA] <= 15
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 5.0) / 2.0
    elseif MINE[row_index, :NA] <= 75
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 15.0) / 3.0
    else
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 45.0) / 4.0
    end

    #println( titlecase(MINE[row_index, :AU]))
    #println(findfirst(primAuthor, titlecase(MINE[row_index, :AU]))[1])
    if findfirst(primAuthor, titlecase(MINE[row_index, :AU]))[1] == 1
        MINE[row_index, :PRIM] = 1
        if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
            MINE[row_index, :PRIM5] = 1
        end
    elseif !ismissing(lucrare[:RP])
        if occursin(primAuthor, lucrare[:RP])
            MINE[row_index, :PRIM] = 1
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
                MINE[row_index, :PRIM5] = 1
            end
        end
    end
    #println(row_index, MINE[row_index, [:NA]])
    #println(row_index, MINE[row_index, [:PRIM]])
end

In [25]:
#VSCodeServer.vscodedisplay(MINE)

In [26]:
#MINE[:, [6, 7, 8, 12, 14, 15, 16, 17]]
MINE.PY = Int64.(MINE.PY)
MINE[ismissing.(MINE[!, :PY]), [1, 6, 7, 8, 12, 14, 15, 16, 17]]

Row,AU,PY,VL,BP,NA,PRIM,AI,AINEFF,IF
,String,Int64,Int64?,String7?,Int64,Int64,Float64,Float64,Float64


In [27]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

In [28]:
AISsubsect = filter(r -> any(occursin.(Vector(MINE[1, [:Journal]]), r.Journal)), AIS)
#subsect2 = filter(r -> (["ADV MATER"][1] .== r.Journal[1]), AIS)
size(AISsubsect, 1)
#["ADV MATER"][1] .== subsect[1, [:Journal]][1]

1

In [29]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    #lucrare = MINE[1, :]
    if(ismissing(lucrare[:Journal]))
        continue
    end
    for AISrow in eachrow(AIS)
        if( lucrare[:Journal] == AISrow[:Journal] )
            if (lucrare[:PY] >= AIFirstYear)
                anString = string(lucrare[:PY])
            else
                anString = string(AIFirstYear)
            end
            if (lucrare[:PY] > AILastYear)
                anString = string(AILastYear)
            end
            #println(AISrow[anString])
            MINE[row_index, :AI] = AISrow[anString]
            MINE[row_index, :AINEFF] = MINE[row_index, :AI] / MINE[row_index, :NEFF]
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
                MINE[row_index, :AINEFF5] = MINE[row_index, :AINEFF]
            end
            
        end
    end
    for ISIrow in eachrow(ISI)
        if( lucrare[:Journal] == ISIrow[:Journal] )
            if (lucrare[:PY] >= AIFirstYear)
                anString = string(lucrare[:PY])
            else
                anString = string(AIFirstYear)
            end
            if (lucrare[:PY] > AILastYear)
                anString = string(AILastYear)
            end
            #println(ISIrow[anString])
            MINE[row_index, :IF] = ISIrow[anString]
            MINE[row_index, :IFNA] = ISIrow[anString] / MINE[row_index, :NA]
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
                MINE[row_index, :IFNA5] = MINE[row_index, :IFNA]
                MINE[row_index, :UAIC1] = (25 + 60*MINE[row_index, :IF]) / MINE[row_index, :NA]
            end
        end
    end
end

In [30]:
#VSCodeServer.vscodedisplay(MINE)
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

In [31]:
I = sum(MINE[!, :AINEFF])

9.68309234099234

In [32]:
P = sum(MINE[!, :PRIM] .* MINE[!, :AI])

18.409

In [33]:
I5 = sum(MINE[!, :AINEFF5])

9.68309234099234

In [34]:
P5 = sum(MINE[!, :PRIM5] .* MINE[!, :AI])

18.409

In [35]:
UAIC_1_1 = sum(MINE[!, :UAIC1])

2622.9813492063495

############################
Citations
############################

In [36]:
#   PT	AU	BA	BE	GP	AF	BF	CA	TI	SO	SE	BS	LA	DT	CT	CY	CL	SP	HO	DE	ID	AB	C1	
#   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  21  22  23  

#   C3	RP	EM	RI	OI	FU	FP	FX	CR	NR	TC	Z9	U1	U2	PU	PI	PA	SN	EI	BN	J9	JI	PD	
#   24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  

#   PY	VL	IS	PN	SU	SI	MA	BP	EP	AR	DI	DL	D2	EA	PG	WC	WE	SC	GA	PM	OA	HC	HP	DA	UT
#   47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71

#   ######  2   9   32  44  46  47  48  54  55  56  57
#   ######  AU  TI  CR  J9  PD  PY  VL  BP  EP  AR  DI

In [37]:
MINECITEraw = CSV.read(inputCIT, DataFrame; header=true, delim='\t');
# autocitari
#for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
#    if( occursin(primAuthor, lucrareCIT.AU) )
#        println(lucrareCIT.AU)
#    end
#end
eltype.(eachcol(MINECITEraw));

In [38]:
#MINECITEraw
#VSCodeServer.vscodedisplay(MINECITEraw)
#inputCIT

In [39]:
# eliminare autocitari
MINECITEraw = filter(row -> !(occursin(primAuthor, row.AU)),  MINECITEraw);
for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
    if( occursin(primAuthor, lucrareCIT.AU) )
        println(lucrareCIT.AU)
    end
end

In [40]:
#unele citari missing PY (early access) - CHECK!!!

#for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
#    if( ismissing(lucrareCIT.PY) )
#        lucrareCIT.PY = 2022
#    end
#end

In [41]:
#VSCodeServer.vscodedisplay(MINECITEraw[1,1:70])

In [42]:
#MINECITE = MINECITEraw[:, [2, 9, 31, 43, 45, 46, 47, 53, 54, 55, 56]];

if usingPART
    ## DEJA SELECTATE COLOANELE in fisierul _PART
    MINECITE = MINECITEraw
else
    MINECITE = MINECITEraw[:, [2, 9, 32, 44, 46, 47, 48, 54, 55, 56, 57]];
end
eltype.(eachcol(MINECITE));
rename!(MINECITE,:"J9" => :"Journal");

In [43]:
#VSCodeServer.vscodedisplay(MINECITE)

In [44]:
#CITES = fill!(Vector{Vector{Int64}}(undef, size(MINE)[1]), [0]);
CITES = Vector{Int64}[];
for i in 1:size(MINE)[1]
    push!(CITES, [0])
end
CITES_IFs = Vector{Float64}[];
for i in 1:size(MINE)[1]
    push!(CITES_IFs, [0.0])
end

In [45]:
CITES5 = Vector{Int64}[];
for i in 1:size(MINE)[1]
    push!(CITES5, [0])
end
CITES5_IFs = Vector{Float64}[];
for i in 1:size(MINE)[1]
    push!(CITES5_IFs, [0.0])
end

In [46]:
for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
    calcul_cit_UAIC = 0.0
    if ( ismissing(lucrareMINE[:DI]) )
        #println("Skip paper $row_indexMINE noDOI")
        continue
    end
    #println("Analyzing paper ", lucrareMINE[:DI])
    for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITE))
        if ( occursin(lucrareMINE[:DI], lucrareCIT[:CR]) )
            #println("Paper $row_indexMINE cited by $row_indexCIT")
            ISI_value = 0.0
            push!(CITES[row_indexMINE], row_indexCIT)
            push!(CITES_IFs[row_indexMINE], ISI_value)
            if (( !ismissing(lucrareCIT[:PY]) ) && ( !ismissing(lucrareCIT[:Journal]) ))
                for ISIrow in eachrow(ISI)
                    if( lucrareCIT[:Journal] == ISIrow[:Journal] )
                        an_publicare_citare = lucrareCIT[:PY]
                        if (an_publicare_citare > AILastYear)
                            an_publicare_citare = AILastYear
                        end
                        if (an_publicare_citare < AIFirstYear)
                            an_publicare_citare = AIFirstYear
                        end
                        ISI_value = ISIrow[string(an_publicare_citare)]
                        push!(CITES_IFs[row_indexMINE], ISI_value)
                        #CITES_IFs[row_indexMINE, size(CITES_IFs[row_indexMINE])] = ISI_value
                        break
                    end
                end
                if (lucrareCIT[:PY] <= ANfin) && (lucrareCIT[:PY] >= ANini)
                    push!(CITES5[row_indexMINE], row_indexCIT)
                    push!(CITES5_IFs[row_indexMINE], ISI_value)
                    calcul_cit_UAIC = calcul_cit_UAIC + (10 + 20 * ISI_value) / lucrareMINE[:NA]
                end
            end
        end
    end
    MINE[row_indexMINE, :TIMESC] = size(CITES[row_indexMINE])[1]-1
    MINE[row_indexMINE, :CT] = MINE[row_indexMINE, :TIMESC] / MINE[row_indexMINE, :NEFF]
    MINE[row_indexMINE, :TIMESC5] = size(CITES5[row_indexMINE])[1]-1
    MINE[row_indexMINE, :CT5] = MINE[row_indexMINE, :TIMESC5] / MINE[row_indexMINE, :NEFF]
    MINE[row_indexMINE, :UAIC9] = calcul_cit_UAIC

end;

In [47]:
CITES5[2]

1-element Vector{Int64}:
 0

In [48]:
size(CITES5_IFs[2])

(1,)

In [49]:
C = sum(MINE[!, :CT])

126.23669663669664

In [50]:
C5 = sum(MINE[!, :CT5])

113.67440892440892

In [51]:
UAIC_1_9 = sum(MINE[!, :UAIC9])

5863.1056746031745

In [52]:
outputMINEcsv = joinpath(root, fileOwnCSV)
CSV.write(outputMINEcsv, MINE)

"/mnt/e/Stoleriu/Doc/CV/auto/MCN_JL_MINE_test.csv"

In [53]:
ENV["COLUMNS"]=500
MINE[1, filter(x -> !(x in ["CR","RP"]), names(MINE))];

In [54]:
using Printf
outputMINEtxt = joinpath(root, fileOwnUAIC)
open(outputMINEtxt, "w") do file
    for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
        if ( !ismissing(lucrareMINE.BP) )
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), pp.$(lucrareMINE.BP)-$(lucrareMINE.EP) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        else
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), art.no.$(lucrareMINE.AR) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        end
        contor = 1
        for (idx_citare, citare) in enumerate(CITES5[row_indexMINE])
            if (citare == 0)
                continue
            end
            write(file, "\t\t[$row_indexMINE.$contor] $(MINECITE[citare,:].AU); $(MINECITE[citare,:].TI); $(MINECITE[citare,:].Journal), vol.$(MINECITE[citare,:].VL) ($(MINECITE[citare,:].PY)), DOI:$(MINECITE[citare,:].DI) \n")
            write(file, "\t\t\tIF citare: $(CITES5_IFs[row_indexMINE][idx_citare])\n")
            contor = contor + 1
        end
        write(file, "nr.autori: $(lucrareMINE.NA) \t\t IF: $(lucrareMINE.IF) \t\t UAIC I.1(ISI): $(@sprintf("%8.4f", lucrareMINE.UAIC1)) \t\t nr.citari: $(lucrareMINE.TIMESC5) \t\t UAIC I.9(Citari): $(@sprintf("%9.4f", lucrareMINE.UAIC9)) \n")
        write(file, '-'^400, '\n')
    end
    write(file, '-'^400, '\n')
    write(file, '-'^400, '\n')
    write(file, "Total criteriul I.1(UAIC) = $(@sprintf("%9.4f", UAIC_1_1))\t\t\tTotal criteriul I.9(UAIC) = $(@sprintf("%9.4f", UAIC_1_9))")
end

77

In [55]:
outputMINEtxt = joinpath(root, fileOwnTXT)
open(outputMINEtxt, "w") do file
    for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
        if ( !ismissing(lucrareMINE.BP) )
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), pp.$(lucrareMINE.BP)-$(lucrareMINE.EP) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        else
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), art.no.$(lucrareMINE.AR) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        end
        contor = 1
        for (idx_citare, citare) in enumerate(CITES5[row_indexMINE])
            if (citare == 0)
                continue
            end
            write(file, "\t\t[$row_indexMINE.$contor] $(MINECITE[citare,:].AU); $(MINECITE[citare,:].TI); $(MINECITE[citare,:].Journal), vol.$(MINECITE[citare,:].VL) ($(MINECITE[citare,:].PY)), DOI:$(MINECITE[citare,:].DI) \n")
            #write(file, "\t\t\tIF citare: $(CITES5_IFs[row_indexMINE][idx_citare])\n")
            contor = contor + 1
        end
        write(file, "autori=$(lucrareMINE.NA) \t autori_eff=$(lucrareMINE.NEFF) \t AIS=$(lucrareMINE.AI) \t prim=$(@sprintf("%5.3f", lucrareMINE.PRIM)) \t inf=$(@sprintf("%5.3f", lucrareMINE.AINEFF)) \t citari=$(lucrareMINE.TIMESC) \t cit=$(@sprintf("%5.3f", lucrareMINE.CT)) \n")
        write(file, '-'^400, '\n')
    end
    write(file, '*'^120, '\n')
    write(file, "P = $(@sprintf("%9.4f", P)) \t\t\t I = $(@sprintf("%9.4f", I)) \t\t\t C = $(@sprintf("%9.4f", C)) \n")
    write(file, '*'^120, '\n')
end

121

In [56]:
outputMINEtxt


"/mnt/e/Stoleriu/Doc/CV/auto/MCN_JL_total.txt"